# Monte Carlo Stability and Convergence

This notebook demonstrates the stability (determinism) and convergence properties of the Monte Carlo engine.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qpl.instruments import EuropeanOption
from qpl.market import Market, FlatRateCurve, FlatDividendCurve
from qpl.models import BlackScholesModel
from qpl.engines.analytic import price_european as bs_price
from qpl.engines.mc.pricers import price_european as mc_price, MCConfig


In [ ]:

# 1. Setup
s0 = 100.0
k = 100.0
t = 1.0
r = 0.05
q = 0.02
sigma = 0.2

option = EuropeanOption(strike=k, expiry=t, kind="call")
market = Market(
    spot=s0, 
    rate_curve=FlatRateCurve(r), 
    dividend_curve=FlatDividendCurve(q)
)
model = BlackScholesModel(sigma=sigma)

ref_val = bs_price(option, model, market).value
print(f"Analytic Price: {ref_val:.6f}")


## Seed Stability
Ensure that re-running with the same seed produces identical results.


In [ ]:

cfg_fixed = MCConfig(n_paths=50000, seed=123)
val1 = mc_price(option, model, market, cfg=cfg_fixed).value
val2 = mc_price(option, model, market, cfg=cfg_fixed).value

print(f"Run 1: {val1:.8f}")
print(f"Run 2: {val2:.8f}")

assert val1 == val2
print("PASS: Results are deterministic given the same seed.")


## Path Convergence
Verify that the price converges to the analytic solution as the number of paths increases, and that the standard error decreases as $1/\sqrt{N}$.


In [ ]:

paths_list = [1000, 2000, 4000, 8000, 16000, 32000, 64000, 128000]
results = []

print("Running MC path scaling...")
for n in paths_list:
    cfg = MCConfig(n_paths=n, seed=42)
    res = mc_price(option, model, market, cfg=cfg)
    
    results.append({
        'N': n,
        'Price': res.value,
        'Stderr': res.stderr,
        'Error': abs(res.value - ref_val)
    })

df = pd.DataFrame(results)
print(df)


In [ ]:

# Visualization

plt.figure(figsize=(12, 5))

# Price vs N with Error Bars
plt.subplot(1, 2, 1)
plt.errorbar(df['N'], df['Price'], yerr=2*df['Stderr'], fmt='o-', capsize=3, label='MC Price (95% CI)')
plt.axhline(ref_val, color='r', linestyle='--', label='Analytic')
plt.xscale('log')
plt.xlabel('Number of Paths')
plt.ylabel('Option Price')
plt.title('MC Convergence')
plt.legend()
plt.grid(True, which="both", alpha=0.5)

# Error Convergence Rate
plt.subplot(1, 2, 2)
# Theoretical decay 1/sqrt(N)
# Clean up slope matching
scale_factor = df['Stderr'].iloc[0] * np.sqrt(df['N'].iloc[0])
theo_curve = scale_factor / np.sqrt(df['N'])

plt.loglog(df['N'], df['Stderr'], 'o-', label='Std Error')
plt.loglog(df['N'], theo_curve, 'k--', label=r'$1/\sqrt{N}$')
plt.xlabel('Number of Paths')
plt.ylabel('Standard Error')
plt.title('Error Decay Rate')
plt.legend()
plt.grid(True, which="both", alpha=0.5)

plt.tight_layout()
plt.show()
